
# Seatfinder – Auswertung aus PostgreSQL-SQL-Dump (2016–2024)

**Hinweis:** Dieses Notebook benötigt **nur die `.sql`-Dump-Datei** (mit `CREATE TABLE` + `COPY ... FROM stdin;` Blöcken). Es verwendet **keine** Datenbankverbindung – der Dump wird direkt geparst und in `pandas`-DataFrames geladen.

**Was du tun musst:**  
1. Lege deine Dump-Datei (z. B. `seatfinder_dump.sql`) im gleichen Ordner ab, in dem du dieses Notebook startest, oder passe unten den Pfad in `SQL_DUMP_PATH` an.  
2. Führe die Zellen nacheinander aus.


# Data Loading to Data Frames


In [6]:
SQL_DUMP_PATH = "data/seatfinder_tuebingen_2024-10-15.sql" 

import os, io, re, csv, datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Matplotlib Defaults – (keine Styles/keine Farben explizit setzen)
plt.rcParams.update({
    "figure.figsize": (9, 4.5),
    "axes.grid": True
})

assert os.path.exists(SQL_DUMP_PATH), f"SQL_DUMP_PATH nicht gefunden: {SQL_DUMP_PATH}"
print("SQL-Dump gefunden:", SQL_DUMP_PATH)

SQL-Dump gefunden: data/seatfinder_tuebingen_2024-10-15.sql



## Dump-Parser (COPY-Blöcke → DataFrames)

Der Parser liest `COPY schema.table (cols...) FROM stdin;` und sammelt die Tab-getrennten Datenzeilen bis zur Zeile `\.`.  
Leere Werte werden als `NaN` gesetzt, `\N` wird ebenfalls zu `NaN`. Datentypen werden später konvertiert.


In [2]:
import io, re
    
COPY_START_RE = re.compile(r'^COPY\s+([a-zA-Z0-9_\."]+)\s*\((.*?)\)\s+FROM\s+stdin;', re.IGNORECASE)
COPY_END = r'\.'

def parse_sql_dump_to_tables(sql_path):
    tables = {}  # key: full table name (e.g., public.locations) -> dict with keys: columns, rows
    current = None
    cols = []
    rows = []
    with io.open(sql_path, "r", encoding="utf-8", errors="ignore") as f:
        for raw_line in f:
            line = raw_line.rstrip("\n")
            if current is None:
                m = COPY_START_RE.match(line)
                if m:
                    current = m.group(1)  # e.g., public.locations
                    col_str = m.group(2)
                    cols = [c.strip().strip('"') for c in col_str.split(",")]
                    rows = []
            else:
                if line == COPY_END:
                    tables.setdefault(current, {"columns": cols, "rows": []})
                    tables[current]["rows"].extend(rows)
                    current = None
                    cols = []
                    rows = []
                else:
                    parts = line.split("\t")
                    if len(parts) < len(cols):
                        parts += [""] * (len(cols) - len(parts))
                    elif len(parts) > len(cols):
                        parts = parts[:len(cols)]
                    parts = [None if p == r"\N" else p for p in parts]
                    rows.append(parts)
    return tables

tables = parse_sql_dump_to_tables(SQL_DUMP_PATH)
print("Found COPY-Tables:", list(tables.keys()))

Found COPY-Tables: ['public.locations', 'public.manual_counts', 'public.seat_estimates', 'public.wlan_clients']



## DataFrames bauen

Wir ziehen die vier relevanten Tabellen in DataFrames:
- `public.locations`
- `public.seat_estimates`
- `public.wlan_clients`
- `public.manual_counts` (falls vorhanden)


In [3]:
def table_to_df(tables, key):
    t = tables.get(key)
    if not t:
        return None
    df = pd.DataFrame(t["rows"], columns=t["columns"])
    return df

locations = table_to_df(tables, "public.locations")
seat_estimates = table_to_df(tables, "public.seat_estimates")
wlan_clients = table_to_df(tables, "public.wlan_clients")
manual_counts = table_to_df(tables, "public.manual_counts")

for name, df in [("locations", locations), ("seat_estimates", seat_estimates),
                 ("wlan_clients", wlan_clients), ("manual_counts", manual_counts)]:
    if df is None:
        print(f"{name}: not found in Dump")
    else:
        print(f"{name}: {df.shape}")

locations: (20, 14)
seat_estimates: (8323622, 5)
wlan_clients: (17489095, 4)
manual_counts: (1177, 5)



## Typen konvertieren

- `timestamp` → `datetime` (naiv, als Europe/Berlin interpretierbar)
- numerische Spalten → `int/float`


In [4]:
def to_datetime_safe(s):
    return pd.to_datetime(s, errors="coerce")

def to_int_safe(s):
    return pd.to_numeric(s, errors="coerce").astype("Int64")

def to_float_safe(s):
    return pd.to_numeric(s, errors="coerce")

# locations
if locations is not None:
    for c in ["id", "availableseats", "superlocation_id"]:
        if c in locations.columns:
            locations[c] = to_int_safe(locations[c])
    if "timestamp" in locations.columns:
        locations["timestamp"] = to_datetime_safe(locations["timestamp"])

# seat_estimates
if seat_estimates is not None:
    for c in ["id", "location_id", "occupiedseats", "freeseats"]:
        if c in seat_estimates.columns:
            seat_estimates[c] = to_int_safe(seat_estimates[c])
    if "timestamp" in seat_estimates.columns:
        seat_estimates["timestamp"] = to_datetime_safe(seat_estimates["timestamp"])

# wlan_clients
if wlan_clients is not None:
    for c in ["id", "location_id", "numberofclients"]:
        if c in wlan_clients.columns:
            wlan_clients[c] = to_int_safe(wlan_clients[c])
    if "timestamp" in wlan_clients.columns:
        wlan_clients["timestamp"] = to_datetime_safe(wlan_clients["timestamp"])

# manual_counts
if manual_counts is not None:
    for c in ["id", "location_id", "occupiedseats", "freeseats"]:
        if c in manual_counts.columns:
            manual_counts[c] = to_int_safe(manual_counts[c])
    if "timestamp" in manual_counts.columns:
        manual_counts["timestamp"] = to_datetime_safe(manual_counts["timestamp"])

# Nützliche Lookup-Infos
if locations is not None:
    loc_lookup = locations.set_index("id")[["name","longname","availableseats"]]
else:
    loc_lookup = pd.DataFrame()

print("Conversion finished.")

Conversion finished.



## CSV-Exporte

Praktisch, wenn man die Aggregationen außerhalb des Notebooks prüfen möchte.


In [5]:
OUTDIR = "exports"
os.makedirs(OUTDIR, exist_ok=True)

if locations is not None: locations.to_csv(os.path.join(OUTDIR, "locations.csv"), index=False)
if seat_estimates is not None: seat_estimates.to_csv(os.path.join(OUTDIR, "seat_estimates.csv"), index=False)
if wlan_clients is not None: wlan_clients.to_csv(os.path.join(OUTDIR, "wlan_clients.csv"), index=False)
if manual_counts is not None: manual_counts.to_csv(os.path.join(OUTDIR, "manual_counts.csv"), index=False)

print("CSV-Exporte in:", OUTDIR)

CSV-Exporte in: exports
